In [1]:
import pandas as pd
import subprocess
import os
print(os.getcwd())

/home/rfpacheco/Documents/Work_CBMSO/Bringaud_testing/2.ingi_in_major/ingi_in_majorV4-0_ws11/1.Analysing_software_results/not_captured


In [2]:
def blastn_dic(path_input, path_output):
    """
    Creation af a BLAST database of our whole genome. It uses the BLAST :sup:`R` command line, see BLAST
    `Command Line Application User Manual`_ for more information.


    The generation of the proper database will be placed in the directory where ``path_input`` is.
    It is recommended to use a dedicated folder to this FASTA file so the database is written next to it.

    :param path_input: path to a FASTA file.
    :type path_input: string

    :param path_output: path to the output folder where the BLAST database will be created.
    :type path_output: string

    :return: a BLAST database.
    :rtype: Muitiples files (**.nhr**, **.nin**, **.nog**, **.nsd**, **.nsi** and **.nsq** extensions)
    """

    # Remember is "path.input.dic_path" for "argparse".
    try:
        # "parse_seqids" is used to keep the sequence ID in the output.
        cmd = f"makeblastdb -in {path_input} -dbtype nucl -parse_seqids -out {path_output}"
        subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception:
        print("\nError: Blast Dictionary couldn't be created")
        

In [3]:
genome_path = "~/Documents/Work_CBMSO/Bringaud_testing/0.1.data/LmjF_V4.0_20040630_SIMPLE_NAMES.fasta"
genome_path = os.path.expanduser(genome_path)
genome_name = os.path.basename(genome_path)
dict_folder = os.path.join(os.path.dirname(genome_path), "blastn_dict_simple_names")
os.makedirs(dict_folder, exist_ok=True)
genome_save_path = os.path.join(dict_folder, genome_name)
# blastn_dic(genome_path, genome_save_path)

In [4]:
def get_sequence(start_coor, end_coor, strand, chromosome, path_genome):
    cmd = f'blastdbcmd -db {path_genome} -entry {chromosome} -range {start_coor}-{end_coor} -strand {strand} -outfmt %s'
    sequence = subprocess.run(cmd, shell=True, capture_output=True, text=True, universal_newlines=True, executable='/usr/bin/bash')
    sequence = sequence.stdout.strip()
    return sequence

In [5]:
not_captured = pd.read_csv('../not_captured-first_output_vs_bringaud.csv', sep=',', header=0)
print(not_captured.shape)
print(not_captured.dtypes)
not_captured.head()

(404, 3)
sseqid    object
sstart     int64
send       int64
dtype: object


,sseqid,sstart,send
0,LmjF.01,226657,226940
1,LmjF.05,90841,91072
2,LmjF.06,57422,57917
3,LmjF.06,61644,62048
4,LmjF.06,178220,178503


In [6]:
# Let's add another column with the number of 'sseqid'
not_captured['chrom_num'] = not_captured['sseqid'].str.split('.').str[1]

# Now transform 'chrom_num' to int64
not_captured['chrom_num'] = pd.to_numeric(not_captured['chrom_num'])

not_captured.head()

,sseqid,sstart,send,chrom_num
0,LmjF.01,226657,226940,1
1,LmjF.05,90841,91072,5
2,LmjF.06,57422,57917,6
3,LmjF.06,61644,62048,6
4,LmjF.06,178220,178503,6


In [7]:
bringaud_siders_complete = pd.read_csv(os.path.expanduser("~/Documents/Work_CBMSO/Bringaud_testing/0.1.data/bringaud_siders.csv"), 
                                       sep=',', header=0)
print(bringaud_siders_complete.shape)
print(bringaud_siders_complete.dtypes)
bringaud_siders_complete.head()

(1852, 8)
ID          object
chr          int64
name        object
fam          int64
str         object
start        int64
end          int64
chr_size     int64
dtype: object


,ID,chr,name,fam,str,start,end,chr_size
0,51.te00022,1,chr1_37.00281,1,-,39705,40532,268984
1,51.te00019,1,chr1_37.00289,1,+,260993,261261,268984
2,51.te00020,1,chr1_37.00290,1,+,266162,266340,268984
3,51.te00018,1,chr1_37.00279,2,+,24174,24821,268984
4,51.te00021,1,chr1_37.00280,2,-,35328,36036,268984


In [8]:
# Perform inner join
not_captured_strand = not_captured.merge(bringaud_siders_complete[['chr', 'start', 'end', 'str']],
                                         left_on=['chrom_num', 'sstart', 'send'],
                                         right_on=['chr', 'start', 'end'],
                                         how='inner')[['sseqid', 'sstart', 'send', 'str']]

# Display results
print(not_captured_strand.shape)
print(not_captured_strand.dtypes)
not_captured_strand.head()

(404, 4)
sseqid    object
sstart     int64
send       int64
str       object
dtype: object


,sseqid,sstart,send,str
0,LmjF.01,226657,226940,-
1,LmjF.05,90841,91072,+
2,LmjF.06,57422,57917,-
3,LmjF.06,61644,62048,-
4,LmjF.06,178220,178503,-


In [9]:
# Rename 'str' to 'sstrand'
not_captured_strand.rename(columns={'str': 'sstrand'}, inplace=True)

# Replace in 'sstrand' '-' with 'minus' and '+' with 'plus'
not_captured_strand['sstrand'] = not_captured_strand['sstrand'].replace('-', 'minus').replace('+', 'plus')

not_captured_strand.head()

,sseqid,sstart,send,sstrand
0,LmjF.01,226657,226940,minus
1,LmjF.05,90841,91072,plus
2,LmjF.06,57422,57917,minus
3,LmjF.06,61644,62048,minus
4,LmjF.06,178220,178503,minus


In [10]:
# Update DataFrame with the 'sseq' column
not_captured_strand['sseq'] = not_captured_strand.apply(
    lambda row: get_sequence(row['sstart'], row['send'], row['sstrand'], row['sseqid'], genome_save_path), 
    axis=1)

print(not_captured_strand.shape)
print(not_captured_strand.dtypes)
not_captured_strand.head()

(404, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LmjF.01,226657,226940,minus,CCACGATGGAGAAGCCAGACGTATGCGCAAGTCGGACGACGGTACC...
1,LmjF.05,90841,91072,plus,CTGGCGATGGTGCGGTGGTGCGTGCCTTGATGAGCAGATGACCCGA...
2,LmjF.06,57422,57917,minus,CTCGGATAAGGTGCGCAGGCACACACACACACACACACACACACAC...
3,LmjF.06,61644,62048,minus,CCGTATGCACACGGGTCTCTCTTGCTGGTGCACCGGCTCTACTACG...
4,LmjF.06,178220,178503,minus,CACCGGTGAGGTGCATCTCCAGCCCGCGCCGCGACTGCGGAGACGC...


In [11]:
# Save data to a CSV
saved_name = "not_captured-first_output_vs_bringaud_WITH_SEQ.csv"
not_captured_strand.to_csv(saved_name, index=False, sep=',', header=True)